
<div class="jumbotron">
  <h1><i class="fa fa-bar-chart" aria-hidden="true"></i> Integracción y limpieza</h1>
  <p></p>
</div>

# Objetivo

Integrar todas las fuentes de datos en un único archivo con formato PICKLE, el cual será utilizado en los pasos de análisis. El archivo se almacena en el folder `data/interim/`.

In [1]:
import pandas as pd
import  pickle
import numpy as np
import glob
import os
import json
import urllib.request
from os import path
#from dotenv import load_dotenv, find_dotenv

os.chdir("..")
pd.set_option('display.width', 1000)

# Obtención de los datos 

# Cargar datos

Leer todos los archivos que fueron descargados a `data/raw/`. Todos deben de contener la misma estructura

### Leer archivo o archivos 

In [2]:
df_1 = pd.read_csv('data/raw/info_01.csv')
df_2 = pd.read_csv('data/raw/info_02.csv')

In [3]:
df_1 = df_1.drop_duplicates(subset=['id', 'id2'])
df_2 = df_2.drop_duplicates(subset=['id', 'id2'])


### Unir los DataFrames

- Ya no existen duplicados

In [4]:
df = pd.merge(df_1, df_2, on=['id', 'id2'], how='inner')

In [5]:
df.head()

,id,id2,v4,v5,v6,v1,c1,v2,v3
0,1,1,721.2,27.3,0.004793,426.0,1,23.2,2015-02-04 17:51:00
1,2,2,714.0,27.3,0.004783,429.5,1,23.1,2015-02-04 17:51:59
2,3,3,713.5,27.2,0.004779,426.0,1,23.1,2015-02-04 17:53:00
3,4,4,708.2,27.2,0.004772,426.0,1,23.1,2015-02-04 17:54:00
4,5,5,704.5,27.2,0.004757,426.0,1,23.1,2015-02-04 17:55:00


### Dimensiones de dataset

In [6]:
df.shape

(8143, 9)

### Inspección de duplicados por pares de IDs

In [7]:
df[df[['id','id2']].sort_values(by=['id','id2']).duplicated(keep=False)].head()

,id,id2,v4,v5,v6,v1,c1,v2,v3


### Comprobar IDs

In [8]:
(df['id'] == df['id2']).all()

True


* `id2`: Entero — Copia de `id`, utilizada para trazabilidad.


In [9]:
df.describe()

,id,id2,v4,v5,v6,v1,c1,v2
count,8143.000000,8143.000000,8143.000000,8143.000000,8143.000000,8143.000000,8143.000000,8143.000000
mean,4072.000000,4072.000000,606.546359,25.731229,0.003863,119.519403,0.212330,20.619821
std,2350.825954,2350.825954,314.320229,5.531708,0.000852,194.755878,0.408982,1.017464
min,1.000000,1.000000,412.800000,16.700000,0.002674,0.000000,0.000000,19.000000
25%,2036.500000,2036.500000,439.000000,20.200000,0.003078,0.000000,0.000000,19.700000
50%,4072.000000,4072.000000,453.500000,26.200000,0.003801,0.000000,0.000000,20.400000
75%,6107.500000,6107.500000,638.850000,30.500000,0.004352,256.400000,0.000000,21.400000
max,8143.000000,8143.000000,2028.500000,39.100000,0.006476,1546.300000,1.000000,23.200000


In [10]:
len(df.id.unique())

8143

In [11]:
len(df.id2.unique())

8143

Las columnas de ID, `id` e `id2`, son **idénticas**. Ambas se removerán en un paso previo

# Descripción de los datos


## Fuentes y diccionario de datos

### Sobre los datos

Los datos provienen de mediciones realizadas por **sensores en una habitación**, registrando si está ocupada o no. 

El dataset incluye columnas de identificación (`id`, `id2`), variables ambientales (`v1`, `v2`, `v4`, `v5`, `v6`), una columna temporal (`v3`) y la variable de clasificación (`c1`).

---

### Diccionario de datos

| Columna | Tipo     | Descripción                                                                                |
| ------- | -------- | ------------------------------------------------------------------------------------------ |
| id      | Entero   | Llave primaria del registro                                                                |
| id2     | Entero   | Repetición de `id` para trazabilidad                                                       |
| v1      | Float    | Luz medida en la unidad lux                                                                |
| v2      | Float    | Temperatura en grados centígrados                                                          |
| v3      | Datetime | Fecha y hora de la medición (`yyyy-MM-dd HH:mm:ss`)                                        |
| v4      | Float    | Cantidad de dióxido de carbono (CO2) en ppm                                                |
| v5      | Float    | Humedad relativa en porcentaje                                                             |
| v6      | Float    | Cociente de humedad, calculado de temperatura y humedad relativa (kg agua-vapor / kg aire) |
| c1      | Entero   | Variable binaria: 0 = cuarto NO ocupado, 1 = ocupado                                       |

El diccionario de datos se encuentra almacenado en la ruta:
`src/conf/data_dict.json`

Este archivo fue generado a partir del **profiling exploratorio y la comprensión del dataset**, y define para cada variable su **nombre representativo, tipo de dato y formato** cuando aplica (por ejemplo, fechas).

| Columna original | Nuevo nombre        |
| ---------------: | ------------------- |
|               id | room_id             |
|              id2 | room_id_duplicate   |
|               v1 | light_lux           |
|               v2 | temperature_celsius |
|               v3 | timestamp           |
|               v4 | co2_ppm             |
|               v5 | humidity_percent    |
|               v6 | humidity_ratio      |
|               c1 | class               |


El diccionario se utilizará posteriormente como **referencia estándar durante la creación del pipeline productivo**, garantizando consistencia en el tipado, formateo y nomenclatura de las columnas a lo largo del flujo de datos.

---

### Transformaciones aplicadas

* Se removieron **duplicados exactos** en todoas las columnas
* Las columnas `id` e `id2` representan la misma información de identificador unico y serán removidas de el dataset.


### Mostrar los 5 primeros renglones

In [12]:
 df.head()

,id,id2,v4,v5,v6,v1,c1,v2,v3
0,1,1,721.2,27.3,0.004793,426.0,1,23.2,2015-02-04 17:51:00
1,2,2,714.0,27.3,0.004783,429.5,1,23.1,2015-02-04 17:51:59
2,3,3,713.5,27.2,0.004779,426.0,1,23.1,2015-02-04 17:53:00
3,4,4,708.2,27.2,0.004772,426.0,1,23.1,2015-02-04 17:54:00
4,5,5,704.5,27.2,0.004757,426.0,1,23.1,2015-02-04 17:55:00


### Mostart las dimensiones del `Dataframe` 

In [13]:
df.shape

(8143, 9)

### Nombre de las columnas 

In [14]:
list(df.columns)

['id', 'id2', 'v4', 'v5', 'v6', 'v1', 'c1', 'v2', 'v3']

### Tipo de datos detectados por Pandas

In [15]:
df.dtypes

id       int64
id2      int64
v4     float64
v5     float64
v6     float64
v1     float64
c1       int64
v2     float64
v3      object
dtype: object

## Dar formato de datetime a las variables de tiempo y fechas 

### Catch string to datetime

In [16]:
# Formatos de fecha comunes
formato= '%Y-%m-%d %H:%M:%S'

# Lista de columnas que quieres convertir
cols = ['v3']
    
df[cols] = df[cols].apply(lambda x: pd.to_datetime(x, format=formato))


### Ortdenar por fecha de registro

In [17]:
df.sort_values(by='v3',inplace=True)

### Valores nulos o faltantes

In [18]:
df.isnull().sum()

id     0
id2    0
v4     0
v5     0
v6     0
v1     0
c1     0
v2     0
v3     0
dtype: int64

### Valores nulos como porcentaje 

In [19]:
((df.isnull().sum() / len(df))*100).sort_values(ascending = False)

id     0.0
id2    0.0
v4     0.0
v5     0.0
v6     0.0
v1     0.0
c1     0.0
v2     0.0
v3     0.0
dtype: float64

<div class="alert alert-info" role="alert">
<ul>
<li> En términos generales, se suelen considerar los siguientes grados de impacto, dependiendo del porcentaje de valores faltantes (dumb rules):

- Menos de 1%: Trivial (no relevante)
- 1-5%: Manejable
- 5-15%: Manejable mediante métodos sofisticados
- Más de 15%: Crítico, con impacto severo en cualquier tipo de interpretación
 
    
Debido a que la mayoría de la variables contiene datos ausentes con un porcentaje inferior al 1% se procederá a remover dichas muestras. La variables 'Genero_Usuario' al no rebasar el 5% de datos ausentes puede continuar dentro de las variables de análisis sin un mayor impacto que remover la muetras que contienen valores nulos.
    
<li> Nota: Por falta de tiempo no se lidia con los datos faltantes de la variable Genero_Usuario. En un entorno productivo si la varible sobre pasa en 1% se debe utilizar tecnicas de inputación de datos para reducir el porcentaje de valores nullos.
    
</div>

## Analisis de valores faltantes

<div class="alert alert-block alert-warning"><b>Atención: </b>
Este notebook sobre cubre el caso base, en el cual,  el porcentaje de valores faltantes es trivial y se eliminan.
Los casos restantes de técnicas de imputación se cubren el el Notebook de referente a trasnformación de datos.

</div>

### Inputación de valores faltantes

In [20]:
df.shape

(8143, 9)

In [21]:
df[df[['id','id2']].sort_values(by=['id','id2']).duplicated(keep='first')]

,id,id2,v4,v5,v6,v1,c1,v2,v3


### Remover identificadores unicos

Dado que los IDs no representan una oportunidad de análisis ni generación de insights, se eliminarán del dataset.

In [22]:
# Lista de columnas a eliminar
cols = ['id', 'id2']

# Eliminar columnas
df = df.drop(columns=cols)

### Valores duplicados

In [23]:
df[df.duplicated(keep='first')]

,v4,v5,v6,v1,c1,v2,v3


### Verificar datos 

In [24]:
# Datos nulos
((df.isnull().sum() / len(df))*100).sort_values(ascending = False)

v4    0.0
v5    0.0
v6    0.0
v1    0.0
c1    0.0
v2    0.0
v3    0.0
dtype: float64

### Convertir las variables a tipo `category`


In [25]:
df['c1'] = df['c1'].astype("category")

In [26]:
df.dtypes

v4           float64
v5           float64
v6           float64
v1           float64
c1          category
v2           float64
v3    datetime64[ns]
dtype: object

### Almacenar el `dataframe` en formato pickle 

El DataFrame se almacena en formato pickle, lo que permite conservar todos los tipos de datos, índices y transformaciones aplicadas durante el preprocesamiento. Esto asegura que en los notebooks posteriores se pueda cargar directamente sin necesidad de volver a verificar o transformar los datos, facilitando la continuidad del análisis y modelado

### **Leer Diccionario de datos**

In [27]:

with open("src/conf/data_dict.json", "r", encoding="utf-8") as f:
    data_dict = json.load(f)


### Renombrar variables

In [28]:
# Crear un mapa de nombres antiguos -> nuevos
rename_map = {old: attrs["nombre"] for old, attrs in data_dict.items()}

# Aplicar rename
df.rename(columns=rename_map, inplace=True)

### Visualizar version final 

In [29]:
df.head()

,co2_ppm,humidity_percent,humidity_ratio,light_lux,class,temperature_celsius,timestamp
0,721.2,27.3,0.004793,426.0,1,23.2,2015-02-04 17:51:00
1,714.0,27.3,0.004783,429.5,1,23.1,2015-02-04 17:51:59
2,713.5,27.2,0.004779,426.0,1,23.1,2015-02-04 17:53:00
3,708.2,27.2,0.004772,426.0,1,23.1,2015-02-04 17:54:00
4,704.5,27.2,0.004757,426.0,1,23.1,2015-02-04 17:55:00


In [30]:
df.shape

(8143, 7)

### Almacenar Dataset

In [31]:
# Store data en formar pickle
with open('data/interim/format.pickle', 'wb') as handle:
    pickle.dump(df, handle)

SyntaxError: invalid syntax (2997548069.py, line 1)